In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.tensor(
    [[0.43, 0.15, 0.89], # Your
     [0.55, 0.87, 0.66], # journey
     [0.57, 0.85, 0.64], # starts
     [0.22, 0.58, 0.33], # with
     [0.77, 0.25, 0.10], # one
     [0.05, 0.80, 0.55]  # step
    ])

In [7]:
query = inputs[2]
attn_scores = torch.zeros(inputs.shape[0])

for i, x in enumerate(inputs):
    attn_scores[i] = torch.dot(x, query)
print(attn_scores)

tensor([0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605])


In [9]:
norm_attn_scores = attn_scores / attn_scores.sum()
print(norm_attn_scores)
print(norm_attn_scores.sum())

tensor([0.1454, 0.2277, 0.2248, 0.1280, 0.1104, 0.1637])
tensor(1.)


In [10]:
def softmax_naive(x):
    exp_x = torch.exp(x)
    return exp_x / exp_x.sum(dim=0)

In [11]:
softmax_attn_score = softmax_naive(attn_scores)
print(softmax_attn_score)
print(softmax_attn_score.sum())

tensor([0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565])
tensor(1.)


In [12]:
softmax_attn_scores = torch.softmax(attn_scores, dim=0)
print(softmax_attn_scores)
print(softmax_attn_scores.sum())

tensor([0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565])
tensor(1.0000)


In [13]:
context_vec = torch.zeros(inputs[0].shape[0])
for i, x in enumerate(inputs):
    context_vec += softmax_attn_scores[i] * x
print(context_vec)

tensor([0.4431, 0.6496, 0.5671])


In [14]:
attn_scores = torch.empty(6, 6)
for i, x in enumerate(inputs):
    for j, y in enumerate(inputs):
        attn_scores[i,j] = torch.dot(x, y)
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [15]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [16]:
attn_scores = torch.softmax(attn_scores, dim=-1)
print(attn_scores)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [17]:
print(attn_scores.sum(dim=-1))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [18]:
context_vecs = attn_scores @ inputs
print(context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [20]:
x_2 = inputs[1]

dim_in, dim_out = inputs.shape[1], 2

In [22]:
torch.manual_seed(123)
W_query = nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_key = nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)
W_value = nn.Parameter(torch.rand(dim_in, dim_out), requires_grad=False)

In [23]:
query_2 = x_2 @ W_query
key_2   = x_2 @ W_key
value_2 = x_2 @ W_value

In [24]:
print(query_2)

tensor([0.4306, 1.4551])


In [26]:
keys   = inputs @ W_key
values = inputs @ W_value

print("key.shape: ", keys.shape)
print("values.shape: ", values.shape)

key.shape:  torch.Size([6, 2])
values.shape:  torch.Size([6, 2])


In [27]:
key_2 = keys[1]
attn_score_22 = query_2.dot(key_2)
print(attn_score_22)

tensor(1.8524)


In [28]:
attn_scores_2 = query_2 @ keys.T
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [29]:
d_k = keys.shape[-1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k ** 0.5, dim=-1) # scaled-dot product attention
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


In [30]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


In [33]:
class SelfAttentionV1(nn.Module):
    def __init__(self, dim_in, dim_out):
        super().__init__()

        self.W_query = nn.Parameter(torch.rand(dim_in, dim_out))
        self.W_key   = nn.Parameter(torch.rand(dim_in, dim_out))
        self.W_value = nn.Parameter(torch.rand(dim_in, dim_out))

    def forward(self, x):
        keys    = x @ self.W_key
        queries = x @ self.W_query
        values  = x @ self.W_value

        attn_scores  = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim = -1)
        context_vec  = attn_weights @ values
        return context_vec

In [34]:
torch.manual_seed(123)
self_attn = SelfAttentionV1(dim_in, dim_out)
print(self_attn(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [37]:
class SelfAttentionV2(nn.Module):
    def __init__(self, dim_in, dim_out, qkv_bias=False):
        super().__init__()

        self.W_query = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_key   = nn.Linear(dim_in, dim_out, bias=qkv_bias)
        self.W_value = nn.Linear(dim_in, dim_out, bias=qkv_bias)

    def forward(self, x):
        queries = self.W_query(x)
        keys = self.W_key(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1] ** 0.5, dim=-1)
        context_vec = attn_weights @ values
        return context_vec

In [39]:
torch.manual_seed(789)
self_attn_2 = SelfAttentionV2(dim_in, dim_out)
print(self_attn_2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [47]:
print(self_attn_2.W_query.weight)

Parameter containing:
tensor([[ 0.3161,  0.4568,  0.5118],
        [-0.1683, -0.3379, -0.0918]], requires_grad=True)


In [54]:
self_attn.W_query.data = self_attn_2.W_query.weight.T
self_attn.W_key.data = self_attn_2.W_key.weight.T
self_attn.W_value.data = self_attn_2.W_value.weight.T

In [55]:
print(self_attn(inputs))
print(self_attn_2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


In [56]:
print(self_attn.W_query.data, self_attn_2.W_query.weight.T)
print(self_attn.W_key.data, self_attn_2.W_key.weight.T)
print(self_attn.W_value.data, self_attn_2.W_value.weight.T)

tensor([[ 0.3161, -0.1683],
        [ 0.4568, -0.3379],
        [ 0.5118, -0.0918]]) tensor([[ 0.3161, -0.1683],
        [ 0.4568, -0.3379],
        [ 0.5118, -0.0918]], grad_fn=<PermuteBackward0>)
tensor([[ 0.4058,  0.2134],
        [-0.4704, -0.2601],
        [ 0.2368, -0.5105]]) tensor([[ 0.4058,  0.2134],
        [-0.4704, -0.2601],
        [ 0.2368, -0.5105]], grad_fn=<PermuteBackward0>)
tensor([[ 0.2526,  0.5191],
        [-0.1415, -0.0852],
        [-0.1962, -0.2043]]) tensor([[ 0.2526,  0.5191],
        [-0.1415, -0.0852],
        [-0.1962, -0.2043]], grad_fn=<PermuteBackward0>)


In [57]:
print(self_attn(inputs))
print(self_attn_2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)
